In [1]:
from copy import deepcopy
from dataclasses import dataclass
from typing import Optional, List, Tuple
import numpy as np
import numpy.random as rnd
import matplotlib.pyplot as plt

from alns import ALNS
from alns.accept import SimulatedAnnealing
from alns.select import AlphaUCB
from alns.stop import MaxIterations, MaxRuntime

# Set random seed for reproducibility
SEED = 2345
rnd.seed(SEED)

@dataclass
class Data:
    n_jobs: int
    n_machines: int
    bkv: int  # best known value
    processing_times: np.ndarray

    @classmethod
    def from_file(cls, path):
        with open(path, "r") as fi:
            lines = fi.readlines()

            n_jobs, n_machines, _, bkv, _ = [
                int(num) for num in lines[1].split()
            ]
            processing_times = np.genfromtxt(lines[3:], dtype=int)

            return cls(n_jobs, n_machines, bkv, processing_times)


def compute_completion_times(schedule):
    """
    Compute the completion time for each job of the passed-in schedule.
    """
    completion = np.zeros(DATA.processing_times.shape, dtype=int)

    for idx, job in enumerate(schedule):
        for machine in range(DATA.n_machines):
            prev_job = completion[machine, schedule[idx - 1]] if idx > 0 else 0
            prev_machine = completion[machine - 1, job] if machine > 0 else 0
            processing = DATA.processing_times[machine, job]

            completion[machine, job] = max(prev_job, prev_machine) + processing

    return completion


def compute_makespan(schedule):
    """
    Returns the makespan, i.e., the maximum completion time.
    """
    return compute_completion_times(schedule)[-1, schedule[-1]]


def plot(schedule, name):
    """
    Plots a Gantt chart of the schedule for the permutation flow shop problem.
    """
    n_machines, n_jobs = DATA.processing_times.shape

    completion = compute_completion_times(schedule)
    start = completion - DATA.processing_times

    # Plot each job using its start and completion time
    cmap = plt.colormaps["rainbow"].resampled(n_jobs)
    machines, length, start_job, job_colors = zip(
        *[
            (i, DATA.processing_times[i, j], start[i, j], cmap(j - 1))
            for i in range(n_machines)
            for j in range(n_jobs)
        ]
    )

    _, ax = plt.subplots(1, figsize=(12, 6))
    ax.barh(machines, length, left=start_job, color=job_colors)

    ax.set_title(f"{name}\n Makespan: {compute_makespan(schedule)}")
    ax.set_ylabel(f"Machine")
    ax.set_xlabel(f"Completion time")
    ax.set_yticks(range(DATA.n_machines))
    ax.set_yticklabels(range(1, DATA.n_machines + 1))
    ax.invert_yaxis()

    plt.show()


class Solution:
    def __init__(
        self, schedule: List[int], unassigned: Optional[List[int]] = None
    ):
        self.schedule = schedule
        self.unassigned = unassigned if unassigned is not None else []

    def objective(self):
        return compute_makespan(self.schedule)

    def insert(self, job: int, idx: int):
        self.schedule.insert(idx, job)

    def opt_insert(self, job: int):
        """
        Optimally insert the job in the current schedule.
        """
        idcs_costs = all_insert_cost(self.schedule, job)
        idx, _ = min(idcs_costs, key=lambda idx_cost: idx_cost[1])
        self.insert(job, idx)

    def remove(self, job: int):
        self.schedule.remove(job)


def all_insert_cost(schedule: List[int], job: int) -> List[Tuple[int, float]]:
    """
    Computes all partial makespans when inserting a job in the schedule.
    O(nm) using Taillard's acceleration. Returns a list of tuples of the
    insertion index and the resulting makespan.

    [1] Taillard, E. (1990). Some efficient heuristic methods for the
    flow shop sequencing problem. European Journal of Operational Research,
    47(1), 65-74.
    """
    k = len(schedule) + 1
    m = DATA.processing_times.shape[0]
    p = DATA.processing_times

    # Earliest completion of schedule[j] on machine i before insertion
    e = np.zeros((m + 1, k))
    for j in range(k - 1):
        for i in range(m):
            e[i, j] = max(e[i, j - 1], e[i - 1, j]) + p[i, schedule[j]]

    # Duration between starting time and final makespan
    q = np.zeros((m + 1, k))
    for j in range(k - 2, -1, -1):
        for i in range(m - 1, -1, -1):
            q[i, j] = max(q[i + 1, j], q[i, j + 1]) + p[i, schedule[j]]

    # Earliest relative completion time
    f = np.zeros((m + 1, k))
    for l in range(k):
        for i in range(m):
            f[i, l] = max(f[i - 1, l], e[i, l - 1]) + p[i, job]

    # Partial makespan; drop the last (dummy) row of q
    M = np.max(f + q, axis=0)

    return [(idx, M[idx]) for idx in np.argsort(M)]


def random_removal(state: Solution, rng, n_remove=2) -> Solution:
    """
    Randomly remove a number jobs from the solution.
    """
    destroyed = deepcopy(state)

    for job in rng.choice(DATA.n_jobs, n_remove, replace=False):
        destroyed.unassigned.append(job)
        destroyed.schedule.remove(job)

    return destroyed


def adjacent_removal(state: Solution, rng, n_remove=2) -> Solution:
    """
    Randomly remove a number adjacent jobs from the solution.
    """
    destroyed = deepcopy(state)

    start = rng.integers(DATA.n_jobs - n_remove)
    jobs_to_remove = [state.schedule[start + idx] for idx in range(n_remove)]

    for job in jobs_to_remove:
        destroyed.unassigned.append(job)
        destroyed.schedule.remove(job)

    return destroyed


def greedy_repair(state: Solution, rng, **kwargs) -> Solution:
    """
    Greedily insert the unassigned jobs back into the schedule. The jobs are
    inserted in non-decreasing order of total processing times.
    """
    state.unassigned.sort(key=lambda j: sum(DATA.processing_times[:, j]))

    while len(state.unassigned) != 0:
        job = state.unassigned.pop()  # largest total processing time first
        state.opt_insert(job)

    return state


def local_search(solution: Solution, **kwargs):
    """
    Improves the current solution in-place using the insertion neighborhood.
    A random job is selected and put in the best new position. This continues
    until relocating any of the jobs does not lead to an improving move.
    """
    improved = True

    while improved:
        improved = False
        current = solution.objective()

        for job in rnd.choice(
            solution.schedule, len(solution.schedule), replace=False
        ):
            solution.remove(job)
            solution.opt_insert(job)

            if solution.objective() < current:
                improved = True
                current = solution.objective()
                break


def greedy_repair_then_local_search(state: Solution, rng, **kwargs):
    """
    Greedily insert the unassigned jobs back into the schedule (using NEH
    ordering). Apply local search afterwards.
    """
    state = greedy_repair(state, rng, **kwargs)
    local_search(state, **kwargs)
    return state


def NEH(processing_times: np.ndarray) -> Solution:
    """
    Schedules jobs in decreasing order of the total processing times.

    [1] Nawaz, M., Enscore Jr, E. E., & Ham, I. (1983). A heuristic algorithm
    for the m-machine, n-job flow-shop sequencing problem. Omega, 11(1), 91-95.
    """
    largest_first = np.argsort(processing_times.sum(axis=0)).tolist()[::-1]
    solution = Solution([largest_first.pop(0)], [])

    for job in largest_first:
        solution.opt_insert(job)

    return solution


In [2]:
def llm_repair(state: Solution, rng, **kwargs) -> Solution:
    """
    Diverse and high-quality repair operators for PFSP using ALNS.
    Inserts unassigned jobs back into the solution in a smart way to minimize makespan.
    """
    # Copy state to avoid in-place modification if needed
    # state = deepcopy(state)  # Uncomment if you want to avoid in-place changes

    # Choose a repair strategy at random (or cycle through them in ALNS)
    strategies = [
        "greedy_insertion",
        "random_insertion",
        "regret_insertion",
        "randomized_greedy",
        "block_insertion"
    ]
    strategy = rng.choice(strategies)

    if strategy == "greedy_insertion":
        # Insert each unassigned job at the position that minimizes makespan
        for job in state.unassigned[:]:
            best_pos = None
            best_makespan = float('inf')
            for idx in range(len(state.schedule) + 1):
                state.insert(job, idx)
                ms = state.objective()
                if ms < best_makespan:
                    best_makespan = ms
                    best_pos = idx
                state.remove(job)
            state.insert(job, best_pos)
            state.unassigned.remove(job)

    elif strategy == "random_insertion":
        # Insert each unassigned job at a random position
        for job in state.unassigned[:]:
            idx = rng.integers(0, len(state.schedule) + 1)
            state.insert(job, idx)
            state.unassigned.remove(job)

    elif strategy == "regret_insertion":
        # Regret-2 insertion: insert the job with the largest regret value
        while state.unassigned:
            best_job = None
            best_regret = -1
            best_pos = None
            for job in state.unassigned:
                costs = []
                for idx in range(len(state.schedule) + 1):
                    state.insert(job, idx)
                    costs.append(state.objective())
                    state.remove(job)
                if len(costs) >= 2:
                    sorted_costs = sorted(costs)
                    regret = sorted_costs[1] - sorted_costs[0]
                else:
                    regret = 0
                if regret > best_regret:
                    best_regret = regret
                    best_job = job
                    best_pos = costs.index(min(costs))
            state.insert(best_job, best_pos)
            state.unassigned.remove(best_job)

    elif strategy == "randomized_greedy":
        # Randomized greedy: for each job, pick one of the k-best positions at random
        k = min(3, len(state.schedule) + 1)
        for job in state.unassigned[:]:
            costs = []
            for idx in range(len(state.schedule) + 1):
                state.insert(job, idx)
                costs.append((state.objective(), idx))
                state.remove(job)
            costs.sort()
            chosen = rng.choice(costs[:k])
            state.insert(job, chosen[1])
            state.unassigned.remove(job)

    elif strategy == "block_insertion":
        # Insert all unassigned jobs as a block at the best position
        block = state.unassigned[:]
        best_pos = None
        best_makespan = float('inf')
        for idx in range(len(state.schedule) + 1):
            for offset, job in enumerate(block):
                state.insert(job, idx + offset)
            ms = state.objective()
            if ms < best_makespan:
                best_makespan = ms
                best_pos = idx
            for job in block:
                state.remove(job)
        for offset, job in enumerate(block):
            state.insert(job, best_pos + offset)
            state.unassigned.remove(job)

    return state

In [3]:
def run_benchmark(data_files, approaches, seed=SEED, iters=8000):
    """
    Benchmark different ALNS approaches on multiple problem instances.
    
    Args:
        data_files: List of paths to problem instance files
        approaches: Dictionary mapping approach names to lists of (destroy_ops, repair_ops)
        seed: Random seed for reproducibility
        iters: Number of iterations for each ALNS run
    
    Returns:
        Dictionary containing all benchmark results
    """
    results = {
        'instance_names': [],
        'instance_sizes': [],
        'problem_types': [],
        'best_known_values': [],
    }
    
    # Initialize results dictionary for each approach
    for approach_name in approaches:
        results[f'{approach_name}_objectives'] = []
        results[f'{approach_name}_gaps'] = []
        results[f'{approach_name}_times'] = []
        results[f'{approach_name}_results'] = []  # Store the ALNS result objects
    
    for data_file in data_files:
        # Extract instance name from file path
        instance_name = data_file.split('/')[-1].split('.')[0]
        problem_type = data_file.split('/')[-2]
        print(f"\nProcessing instance: {instance_name} (Type: {problem_type})")
        
        # Load data
        data = Data.from_file(data_file)
        global DATA  # Use global DATA variable for the operators
        DATA = data
        
        results['instance_names'].append(instance_name)
        results['problem_types'].append(problem_type)
        results['instance_sizes'].append(f"{data.n_jobs}x{data.n_machines}")
        results['best_known_values'].append(data.bkv)
        
        # Create initial solution using NEH
        init = NEH(data.processing_times)
        
        # Run each approach
        for approach_name, (destroy_ops, repair_ops) in approaches.items():
            print(f"  Running {approach_name}...")
            
            # Setup ALNS
            alns = ALNS(rnd.default_rng(seed))
            
            # Add destroy operators
            for destroy_op in destroy_ops:
                alns.add_destroy_operator(destroy_op)
            
            # Add repair operators
            for repair_op in repair_ops:
                alns.add_repair_operator(repair_op)
            
            # Configure ALNS parameters
            select = AlphaUCB(
                scores=[5, 2, 1, 0.5],
                alpha=0.05,
                num_destroy=len(alns.destroy_operators),
                num_repair=len(alns.repair_operators),
            )
            accept = SimulatedAnnealing.autofit(init.objective(), 0.05, 0.50, iters)
            stop = MaxIterations(iters)
            
            # Add time tracking
            time_stop = MaxRuntime(3600)  # 1 hour max runtime
            
            # Run ALNS
            start_time = time.time()
            result = alns.iterate(deepcopy(init), select, accept, stop)
            runtime = time.time() - start_time
            
            # Record results
            objective = result.best_state.objective()
            gap = 100 * (objective - data.bkv) / data.bkv
            
            results[f'{approach_name}_objectives'].append(objective)
            results[f'{approach_name}_gaps'].append(gap)
            results[f'{approach_name}_times'].append(runtime)
            results[f'{approach_name}_results'].append(result)  # Store the ALNS result object
            
            print(f"    Objective: {objective}, Gap: {gap:.2f}%, Time: {runtime:.2f}s")
    
    return results

def visualize_results(results):
    """
    Create visualizations to compare the approaches.
    """
    approach_names = [name.split('_')[0] for name in results.keys() 
                     if name.endswith('_objectives')]
    
    # Create a DataFrame for easier manipulation
    df = pd.DataFrame({
        'Instance': results['instance_names'],
        'Problem_Type': results['problem_types'],
        'Size': results['instance_sizes'],
        'BKV': results['best_known_values']
    })
    
    for approach in approach_names:
        df[f'{approach}_Obj'] = results[f'{approach}_objectives']
        df[f'{approach}_Gap'] = results[f'{approach}_gaps']
        df[f'{approach}_Time'] = results[f'{approach}_times']
    
    # 1. Problem type average gap comparison
    plt.figure(figsize=(14, 8))
    
    # Calculate average gap by problem type
    problem_type_avg = df.groupby('Problem_Type').apply(
        lambda x: pd.Series({
            f"{approach}_AvgGap": x[f"{approach}_Gap"].mean() 
            for approach in approach_names
        })
    ).reset_index()
    
    # Prepare data for bar chart
    problem_types = problem_type_avg['Problem_Type']
    x = np.arange(len(problem_types))
    width = 0.8 / len(approach_names)
    
    for i, approach in enumerate(approach_names):
        plt.bar(x + i*width - 0.4 + width/2, problem_type_avg[f'{approach}_AvgGap'], 
                width=width, label=approach)
    
    plt.xlabel('Problem Type')
    plt.ylabel('Average Gap to BKV (%)')
    plt.title('Average Performance Gap by Problem Type')
    plt.xticks(x, problem_types, rotation=45)
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()
    
    # 2. Problem type average runtime comparison
    plt.figure(figsize=(14, 8))
    
    # Calculate average runtime by problem type
    problem_type_avg_time = df.groupby('Problem_Type').apply(
        lambda x: pd.Series({
            f"{approach}_AvgTime": x[f"{approach}_Time"].mean() 
            for approach in approach_names
        })
    ).reset_index()
    
    for i, approach in enumerate(approach_names):
        plt.bar(x + i*width - 0.4 + width/2, problem_type_avg_time[f'{approach}_AvgTime'], 
                width=width, label=approach)
    
    plt.xlabel('Problem Type')
    plt.ylabel('Average Runtime (seconds)')
    plt.title('Average Runtime by Problem Type')
    plt.xticks(x, problem_types, rotation=45)
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()
    
    # 3. Summary table by problem type
    summary = pd.DataFrame({
        'Problem_Type': problem_types,
    })
    
    for approach in approach_names:
        summary[f'{approach}_AvgGap'] = problem_type_avg[f'{approach}_AvgGap']
    
    display(summary)
    
    # 4. Overall summary
    overall_summary = pd.DataFrame({
        'Approach': approach_names,
        'Avg Gap (%)': [df[f'{approach}_Gap'].mean() for approach in approach_names],
        'Best Gap (%)': [df[f'{approach}_Gap'].min() for approach in approach_names],
        'Worst Gap (%)': [df[f'{approach}_Gap'].max() for approach in approach_names],
        'Avg Time (s)': [df[f'{approach}_Time'].mean() for approach in approach_names],
    })
    
    display(overall_summary)
    
    # 5. Detailed results table
    display(df)
    
    return df

if __name__ == "__main__":
    import time
    import pandas as pd
    
    # Define the approaches to compare
    approaches = {
        'Baseline': (
            [random_removal, adjacent_removal],  # destroy operators
            [greedy_repair]    # repair operators
        ),
        'CHATGPT': (
            [random_removal, adjacent_removal],  # destroy operators
            [llm_repair]    # repair operators
        )
    }
    
    # List all relevant Taillard instances - 10 instances for each problem type up to j100_m10
    data_files = []
    
    # j20_m5 (10 instances)
    for i in range(1, 11):
        data_files.append(f"data/j20_m5/j20_m5_{i:02d}.txt")
        
    # j20_m10 (10 instances)
    for i in range(1, 11):
        data_files.append(f"data/j20_m10/j20_m10_{i:02d}.txt")
        
    # j20_m20 (10 instances)
    for i in range(1, 11):
        data_files.append(f"data/j20_m20/j20_m20_{i:02d}.txt")
        
    # j50_m5 (10 instances)
    for i in range(1, 11):
        data_files.append(f"data/j50_m5/j50_m5_{i:02d}.txt")
        
    # j50_m10 (10 instances)
    for i in range(1, 11):
        data_files.append(f"data/j50_m10/j50_m10_{i:02d}.txt")
        
    # j50_m20 (10 instances)
    for i in range(1, 11):
        data_files.append(f"data/j50_m20/j50_m20_{i:02d}.txt")
        
    # j100_m5 (10 instances)
    for i in range(1, 11):
        data_files.append(f"data/j100_m5/j100_m5_{i:02d}.txt")
        
    # j100_m10 (10 instances)
    for i in range(1, 11):
        data_files.append(f"data/j100_m10/j100_m10_{i:02d}.txt")
    
    # Run the benchmark
    results = run_benchmark(data_files, approaches, seed=SEED, iters=600)
    
    # Visualize and analyze the results
    results_df = visualize_results(results)
    
    # Save results to CSV
    results_df.to_csv('alns_operator_selection_results.csv', index=False)


Processing instance: j20_m5_01 (Type: j20_m5)
  Running Baseline...
    Objective: 1286, Gap: 0.63%, Time: 0.38s
  Running CHATGPT...
    Objective: 1286, Gap: 0.63%, Time: 1.12s

Processing instance: j20_m5_02 (Type: j20_m5)
  Running Baseline...
    Objective: 1360, Gap: 0.07%, Time: 0.38s
  Running CHATGPT...
    Objective: 1365, Gap: 0.44%, Time: 1.12s

Processing instance: j20_m5_03 (Type: j20_m5)
  Running Baseline...
    Objective: 1087, Gap: 0.56%, Time: 0.39s
  Running CHATGPT...
    Objective: 1100, Gap: 1.76%, Time: 1.27s

Processing instance: j20_m5_04 (Type: j20_m5)
  Running Baseline...
    Objective: 1299, Gap: 0.46%, Time: 0.37s
  Running CHATGPT...
    Objective: 1306, Gap: 1.01%, Time: 1.10s

Processing instance: j20_m5_05 (Type: j20_m5)
  Running Baseline...
    Objective: 1235, Gap: -0.08%, Time: 0.37s
  Running CHATGPT...
    Objective: 1250, Gap: 1.13%, Time: 1.10s

Processing instance: j20_m5_06 (Type: j20_m5)
  Running Baseline...
    Objective: 1207, Gap: 1.00

KeyboardInterrupt: 